In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gzip
import os

from NREL_processing import get_data

In [87]:
API_URL = "https://developer.nrel.gov/api/alt-fuel-stations/v1.csv"
API_PARAMS = {'api_key': 'EUe0n9InavfhKtKtmscW1Emd5b3IhaJwOkcHu3MN', 'fuel_type': 'ELEC'}
RAW_FILE = "../data/NREL_raw.csv"

In [88]:
if not os.path.exists(RAW_FILE):
    get_data(API_URL, API_PARAMS, RAW_FILE)
raw_data = pd.read_csv(RAW_FILE, delimiter='\t')

/tmp/ipykernel_39230/1608278770.py:3: DtypeWarning: Columns (6,20,31,46,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(RAW_FILE, delimiter='\t')


In [89]:
EV_columns = [
    'Station Name', 'Street Address', 'Intersection Directions',
    'City', 'State', 'ZIP', 'Plus4', 'Station Phone',
    'Access Days Time', 'Cards Accepted',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
    'EV Other Info', 'EV Network', 'EV Network Web',
    'Geocode Status', 'Latitude', 'Longitude',
    'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
    'Federal Agency ID', 'Federal Agency Name', 'Open Date',
    'EV Connector Types', 'Country', 'Access Code', 'Access Detail Code',
    'Federal Agency Code', 'Facility Type',
    'EV Pricing', 'EV On-Site Renewable Source', 'Restricted Access',
    'NPS Unit Name', 'Maximum Vehicle Class', 'EV Workplace Charging'
]

In [90]:
raw_data = raw_data[raw_data['Status Code'] == 'E']
raw_data = raw_data[EV_columns]
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64894 entries, 0 to 69579
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Station Name                 64893 non-null  object 
 1   Street Address               64861 non-null  object 
 2   Intersection Directions      2380 non-null   object 
 3   City                         64890 non-null  object 
 4   State                        64881 non-null  object 
 5   ZIP                          64894 non-null  object 
 6   Plus4                        0 non-null      float64
 7   Station Phone                61208 non-null  object 
 8   Access Days Time             57894 non-null  object 
 9   Cards Accepted               5959 non-null   object 
 10  EV Level1 EVSE Num           673 non-null    float64
 11  EV Level2 EVSE Num           56564 non-null  float64
 12  EV DC Fast Count             9216 non-null   float64
 13  EV Other Info        

In [91]:
str_columns = [
    'Street Address', 'Intersection Directions', 'City',
    'State', 'ZIP', 'Plus4', 'Station Phone', 'Access Days Time', 
    'EV Other Info', 'EV Network', 'EV Network Web', 
    'EV Connector Types', 'Country', 'Access Code', 'Access Detail Code', 
    'Facility Type', 'EV Pricing', 'EV On-Site Renewable Source',
    'NPS Unit Name', 
]

enum_columns = [
    'Cards Accepted', 'Owner Type Code', 
    'Federal Agency Name', 'Federal Agency Code',
    'Maximum Vehicle Class'
]

num_columns = [
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
    'Latitude', 'Longitude', 'ID', 'Federal Agency ID'
]


In [92]:
vehicle_class_map = {
    'LD': 'Passenger vehicles (class 1-2)',
    'MD': 'Medium-duty (class 3-5)',
    'HD': 'Heavy-duty (class 6-8)'
}

owners_map = {
    'FG': 'Federal Government Owned',
    'J': 'Jointly Owned',
    'LG': 'Local/Municipal Government Owned',
    'P': 'Privately Owned',
    'SG': 'State/Provincial Government Owned',
    'T': 'Utility Owned'
}

cards_map = {
    'A': 'American Express',
    'C': 'Credit',
    'Debit': 'Debit',
    'D': 'Discover',
    'M': 'MasterCard',
    'V': 'Visa',
    'Cash': 'Cash',
    'Checks': 'Checks',
    'ACCOUNT_BALANCE': 'Account Balance',
    'ALLIANCE': 'Alliance AutoGas',
    'ANDROID_PAY': 'Android Pay',
    'APPLE_PAY': 'Apple Pay',
    'ARI': 'ARI',
    'CleanEnergy': 'Clean Energy',
    'Comdata': 'Comdata',
    'CFN': 'Commercial Fueling Network',
    'EFS': 'EFS',
    'FleetOne': 'Fleet One',
    'FuelMan': 'Fuelman',
    'GasCard': 'GASCARD',
    'PacificPride': 'Pacific Pride',
    'PHH': 'PHH',
    'Proprietor': 'Proprietor Fleet Card',
    'Speedway': 'Speedway',
    'SuperPass': 'SuperPass',
    'TCH': 'TCH',
    'Tchek': 'T-Chek T-Card',
    'Trillium': 'Trillium',
    'Voyager': 'Voyager',
    'Wright_Exp': 'WEX'
}

In [93]:
def make_jsonl():
    raw_data.fillna('', inplace=True)
    # raw_data['Cards Accepted'].replace(cards_map, inplace=True)
    # raw_data['Owner Type Code'].replace(owners_map, inplace=True)
    # raw_data['Maximum Vehicle Class'].replace(vehicle_class_map, inplace=True)
    str_data = raw_data[str_columns + enum_columns]
    concatenated = str_data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df_conc = pd.DataFrame(concatenated, columns=['text'])
    df_conc['docid'] = raw_data['ID'].astype(str)
    df_conc['name'] = raw_data['Station Name']
    df_conc = df_conc[['docid', 'name', 'text']]
    df_conc.to_json('../data/NREL_corpus.jsonl', lines=True, orient='records')

make_jsonl()

/tmp/ipykernel_39230/3603274366.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  raw_data.fillna('', inplace=True)


In [94]:
def plot_states():
    states_grouped = raw_data.groupby('State').count().reset_index()
    states_grouped = states_grouped[['State', 'Status Code']]
    states_sorted = states_grouped.sort_values(by='Status Code', ascending=False)
    states_sorted = states_sorted.reset_index(drop=True)
    plt.figure(figsize=(15,10))
    hist_plot = sns.countplot(x='State', data=raw_data, order=states_sorted['State'])
    hist_plot.set_xticklabels(hist_plot.get_xticklabels(), rotation=90)
    hist_plot.set_title('Number of Charging Stations by State')
    plt.show()

# plot_states()

In [95]:
def plot_vehicles():
    hist = sns.histplot(data=raw_data, x='Maximum Vehicle Class')
    hist.set_title('Number of Charging Stations by Maximum Vehicle Class')
    hist.set_yscale('log')
    plt.show()

# plot_vehicles()

In [96]:
def plot_facilities():
    facility_grouped = raw_data.groupby('Facility Type').count().reset_index()
    facility_grouped = facility_grouped[['Facility Type', 'Status Code']]
    facility_sorted = facility_grouped.sort_values(by='Status Code', ascending=False)
    facility_sorted = facility_sorted.reset_index(drop=True)
    facility_sorted = facility_sorted[:10]
    hist = sns.countplot(data=raw_data, x='Facility Type', order=facility_sorted['Facility Type'])
    hist.set_title('Number of Charging Stations by Top 10 Facility Type')
    hist.set_xticklabels(hist.get_xticklabels(), rotation=90)
    plt.show()

# plot_facilities()

In [97]:
def plot_types():
    fig, ax = plt.subplots(3, 1, figsize=(10, 5))
    plt.subplots_adjust(top=2)
    ax = ax.flatten()
    bins = range(0, 150, 10)
    sns.histplot(data=raw_data, x='EV Level1 EVSE Num', bins=bins, ax=ax[0])
    ax[0].set_yscale('log')
    ax[0].set_ylim(1, 100000)
    ax[0].set_title('Number of Charging Stations by Type')
    sns.histplot(data=raw_data, x='EV Level2 EVSE Num', bins=bins, ax=ax[1])
    ax[1].set_yscale('log')
    ax[1].set_ylim(1, 100000)
    sns.histplot(data=raw_data, x='EV DC Fast Count', bins=bins, ax=ax[2])
    ax[2].set_yscale('log')
    ax[2].set_ylim(1, 100000)

# plot_types()

In [98]:
google_data = pd.read_csv('./data/Google_Map_review_data.csv')
google_data.lat = google_data.lat.astype(float)
google_data.lng = google_data.lng.astype(float)

FileNotFoundError: [Errno 2] No such file or directory: './data/Google_Map_review_data.csv'

In [ ]:
mask = (abs(raw_data.Latitude - google_data.lat[0]) < 0.001) & (abs(raw_data.Longitude - google_data.lng[0]) < 0.001)

In [ ]:
selected = raw_data[mask]
selected

,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,EV Level2 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
11784,"Walmart 2508 - Boise, ID",8300 W. Overland Rd,Boise,ID,E,Public,24 hours daily,NaN,NaN,NaN,...,43.59142,-116.28359,NaN,2019-04-26,CHADEMO J1772COMBO,NaN,NaN,NaN,NaN,False


In [ ]:
raw_data = raw_data[raw_data['Status Code'] == 'E']
raw_data.reset_index(drop=True, inplace=True)
raw_data['docid'] = raw_data.index + 1
raw_data = raw_data[['docid'] + test_params]

In [ ]:
state = 'NY'
raw_state = raw_data[raw_data['State'] == state]
raw_state['Street Address'].fillna('None', inplace=True)
raw_state[raw_state['Street Address'].str.contains('Conserva')]
# aa[aa['Station Name'].str.contains('AIRPORT')]

/tmp/ipykernel_7991/475096227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_state['Street Address'].fillna('None', inplace=True)


,docid,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
18536,18537,ITHACA VISITORS ITHACA COLLEGE,Conservatory Dr,Ithaca,NY,E,Public,24 hours daily,NaN,NaN,...,42.420935,-76.500084,NaN,2020-06-12,J1772,NaN,NaN,NaN,NaN,False


In [ ]:
raw_data[raw_data['docid'] == 6702]

,docid,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
6701,6702,Meijer - Tesla Supercharger,3145 Ann Arbor-Saline Road,Ann Arbor,MI,E,Public,24 hours daily,NaN,NaN,...,42.241125,-83.766522,P,2015-06-01,TESLA,NaN,NaN,NaN,NaN,False
